# Métrique NLI


In [15]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px
import re
from difflib import SequenceMatcher

# **I - Chargement des données**

In [16]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

# Ajout de nouvelles colonnes
# Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

# Copie des colonnes d'hallucinations/d'idées invalides en version catégorielle
mapping = {0:"Non", 1:"Oui"}
df["pres_hallu"] = df["Hallucinations"].map(mapping)
df["pres_idees_inv"] = df["Idées_non_ind"].map(mapping)

# Nombre de tokens par contribution et par extractions (comptage simple par espaces)
df["nb_tokens_contrib"] = df["contribution"].apply(lambda x: len(str(x).split()))
df["nb_tokens_extraction"] = df["ideas_text"].apply(lambda x: len(str(x).split()))

# **II - Présentation et évaluation de la métrique**

La métrique NLI ( Natural Language Inference ) est une métrique qui vise à mesurer si une hypothèse $h$ est induite par une prémisse $p$, qu'elle soit contradictoire ou bien neutre.

Pour ce faire, on va définir :
- les prémises $p$ comme nos contributions
- Les hypothèses $h$ comme les idées extraites par le LLM

L'idée de la métrique est la suivante :
Pour chaques idées $h_i$, nous calculons le meilleur score de similarité avec la phrase de contribution. Nous utilisons un mélange de deux méthodes pour cela :
-*Jaccard tokenks* : C'est un recouvrement du vocbulaire
-*LCS ratio* : C'est une métrique ROUGE-L simplifiée
- On calcule ensuite la combinaison suivante :

$$ support(h_i) = max_j(0.6Jaccard(h_i,p_j)+0.4LCS(h_i,p_j))$$


Ensuite, on détecte la présence de négation dans $h_i$ et $p_j$>
Si la meilleure phrase $p_j$ ( celle qui maximise le support de $support(h_i)$) mais dispose d'une négation différente de $h_i$, on ajoute une pénalité sur la métrique NLI en fonction d'un facteur $\alpha \in ]0,1[$ :

$$
\mathrm{contra}(h_i) =
\mathbf{1}\!\left[\mathrm{neg}(h_i)\neq \mathrm{neg}\!\left(p_{j^\ast}\right)\right]
\cdot
\left(1-\mathrm{support}(h_i)\right)
\cdot
\alpha
$$




Pour obtenir le score final de la métrique, nous moyennenons sur toutes les idées :
- *NLI_support* : varie de 0 à 1. Plus nous sommes proches de ! et plus la métrique est entrainée (bonne)
- *NLI_contra* : varie de 0 à 1. Plus nous somme proches de 1 et plus les idées entre extractions et contributions sont opposées
- On calcule *NLI_final* comme la différence de NLI_support et NLI_contra

In [17]:
# Jaccard et LCS ratio
_SENT_SPLIT = re.compile(r'(?<=[\.\?\!])\s+|\n+')
_WORD = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9']+")

NEG_MARKERS = {
    "ne", "n", "pas", "plus", "jamais", "aucun", "aucune", "sans", "ni", "rien", "personne"
}

STOPWORDS_FR_MINI = {
    # mini stoplist (évite de dépendre d'un package)
    "le","la","les","un","une","des","du","de","d","et","ou","à","a","au","aux",
    "en","dans","sur","pour","par","avec","sans","ce","cet","cette","ces",
    "que","qui","quoi","dont","où","est","sont","être","été","être","il","elle",
    "ils","elles","on","nous","vous","je","tu","se","sa","son","ses","leur","leurs",
    "mais","donc","car","si","comme","plus","moins","très"
}

def split_sentences(text: str):
    if not isinstance(text, str) or not text.strip():
        return []
    text = re.sub(r"\s+", " ", text.strip())
    return [s.strip() for s in _SENT_SPLIT.split(text) if s and s.strip()]

def tokenize(text: str):
    if not isinstance(text, str):
        return []
    toks = [t.lower() for t in _WORD.findall(text)]
    return toks

def content_tokens(text: str):
    toks = tokenize(text)
    return [t for t in toks if t not in STOPWORDS_FR_MINI and len(t) > 2]

def has_negation(text: str):
    toks = tokenize(text)
    return any(t in NEG_MARKERS for t in toks) or "n'" in text.lower()

def jaccard(a_tokens, b_tokens):
    A, B = set(a_tokens), set(b_tokens)
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

def lcs_ratio(a: str, b: str):
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def parse_ideas_text(ideas_text: str):
    if not isinstance(ideas_text, str) or not ideas_text.strip():
        return []
    ideas = split_sentences(ideas_text)
    ideas = [x.strip() for x in ideas if len(x.strip()) > 3]
    return ideas



In [18]:
# Score NLI

def nli_lexical_scores(premise: str, ideas_text: str, alpha_contra: float = 0.8):
    premise_sents = split_sentences(premise)
    if not premise_sents:
        return np.nan, np.nan, np.nan

    ideas = parse_ideas_text(ideas_text)
    if not ideas:
        return np.nan, np.nan, np.nan

    contra_scores = []
    support_scores = []

    for h in ideas:
        h_tok = content_tokens(h)
        h_neg = has_negation(h)

        best_support = 0.0
        best_sent = None

        for ps in premise_sents:
            ps_tok = content_tokens(ps)
            s_j = jaccard(h_tok, ps_tok)
            s_l = lcs_ratio(h, ps)
            support = 0.6 * s_j + 0.4 * s_l
            if support > best_support:
                best_support = support
                best_sent = ps

        support_scores.append(best_support)

        if best_sent is None:
            contra_scores.append(0.0)
        else:
            ps_neg = has_negation(best_sent)
            mismatch = 1.0 if (h_neg != ps_neg) else 0.0
            contra = mismatch * (1.0 - best_support) * alpha_contra
            contra_scores.append(contra)

    support_mean = float(np.mean(support_scores)) if support_scores else np.nan
    contra_mean = float(np.mean(contra_scores)) if contra_scores else np.nan
    final = float(np.clip(support_mean - contra_mean, 0.0, 1.0)) if np.isfinite(support_mean) and np.isfinite(contra_mean) else np.nan

    return support_mean, contra_mean, final


In [19]:
# Application à nos données
df["NLI_support"] = np.nan
df["NLI_contra"] = np.nan
df["NLI_final"] = np.nan

for i in range(len(df)):
    premise = str(df.loc[i, "contribution"]) if "contribution" in df.columns else ""
    ideas_text = str(df.loc[i, "ideas_text"]) if "ideas_text" in df.columns else ""
    s, c, f = nli_lexical_scores(premise, ideas_text, alpha_contra=0.8)
    df.loc[i, "NLI_support"] = s
    df.loc[i, "NLI_contra"] = c
    df.loc[i, "NLI_final"] = f

df[["NLI_support", "NLI_contra", "NLI_final"]].describe()

,NLI_support,NLI_contra,NLI_final
count,200.000000,200.000000,200.000000
mean,0.420649,0.162663,0.350180
std,0.251018,0.263954,0.294073
min,0.000000,0.000000,0.000000
25%,0.221919,0.000000,0.060260
50%,0.401881,0.000000,0.322630
75%,0.567480,0.326125,0.537564
max,1.000000,0.773922,1.000000


### Remarques

L’article SelfCheckGPT propose une variante SelfCheckGPT-NLI basée sur un classifieur NLI  afin d’estimer une probabilité de contradiction entre une phrase et un contexte. Dans l’implémentation, cela nécessite l’utilisation de bibliothèques de deep learning ainsi que le téléchargement et l’exécution d’un modèle pré-entraîné.

Dans ce projet, l’objectif est de conserver une métrique indépandante du LLM.

De plus, notre environnement d’exécution ne permet pas d’installer correctement `torch`/`transformers` (python 3.13 pour ma part). Pour ces raisons, nous n’implémentons pas la version “modèle NLI” de SelfCheckGPT, mais une approximation NLI lexicale calculée à la main comme pour la métrique QualIT.


Cette métrique doit être interprétée comme une métrique basée sur celle NLI, permettant une comparaison directe avec QualIT et ROUGE dans un cadre ou on peut comparer les scores des métriques d'un LLM à l'autre.


In [20]:
correlation_nli = df["score_humain"].corr(df["NLI_final"], method="pearson")
print(f"Corrélation (Pearson) entre score humain moyen et métrique NLI_final : {correlation_nli:.2f}")

Corrélation (Pearson) entre score humain moyen et métrique NLI_final : 0.52


La corrélation de Pearson affiche un score de cohérence entre l'évaluation humaine et la métrique NLI de 0.52 : c'est le score le plus faible obtenu jusqu'à maintenant

In [21]:

fig = px.density_contour(
    df, x="NLI_final", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels=False, colorscale="Blues")

# Scatter plot
fig.add_scatter(
    x=df["NLI_final"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique NLI_final = %{x}<br>Score humain = %{y}<extra></extra>"
)

# Ligne de référence (x = y) adaptée aux échelles : x∈[0,1], y∈[0,10]
# => y = 10x
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)

fig.update_layout(
    # title="<b>Score humain vs métrique NLI_final</b>",
    xaxis_title="NLI_final metric", yaxis_title="Human score",
    width=700, height=500,
    coloraxis_showscale=False
)

fig.show()


En terme de détection d'hallucination on est vraiment pas mal. Quelques unes ne sont pas détectées mais globalement on les capte bien.

A ajouter : Comparaison NLI vs Score humain avec colloration selon :
- présence idées séparées
- présence d'hallucinations
- longueur de la contribution parmi les 4 catégories 
(Voir 02_MetriqueQualIT)